In [1]:
import os
import json
import glob
import numpy as np
from pathlib import Path

import pandas as pd
import pyarrow as pa

from fnmatch import fnmatch 

import tqdm

from natsort import natsorted, index_natsorted

In [2]:
import sys
from importlib import reload

sys.path.insert(0, '..')

import pali_text_models.models
reload(pali_text_models.models)

import pali_text_models
reload(pali_text_models)

from pali_text_models import get_session, SuttaTextModel



/Users/oceansxyz/.local/share/uv/tools/jupyter-core/lib/python3.12/site-packages/sqlmodel/main.py:682: SAWarning: This declarative base already contains a class with the same class name and module name as pali_text_models.models.SuttaTextModel, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)
/Users/oceansxyz/.local/share/uv/tools/jupyter-core/lib/python3.12/site-packages/sqlmodel/main.py:682: SAWarning: This declarative base already contains a class with the same class name and module name as pali_text_models.models.AbhidhammaTextModel, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)


In [11]:
from sqlmodel import SQLModel, Field, create_engine, Session
from typing import Optional, List, Dict
from sqlalchemy.dialects.postgresql import JSONB, ARRAY, FLOAT
from sqlalchemy.engine import Engine

In [4]:
df = pd.read_csv('../pali_canon_hela.csv', index_col=0)

df.head()

,nikaya,vagga,sutta_id,text,hela_text
index,,,,,
dn1:0.1,dn,1,dn1,Dīgha Nikāya 1,දීඝ නිකාය 1
dn1:0.2,dn,1,dn1,Brahmajālasutta,බ්‍රහ්මජාලසුත්ත
dn1:1.0,dn,1,dn1,1. Paribbājakakathā,1. පරිබ්බාජකකථා
dn1:1.1.1,dn,1,dn1,Evaṁ me sutaṁ—,ඒවං මේ සුතං—
dn1:1.1.2,dn,1,dn1,ekaṁ samayaṁ bhagavā antarā ca rājagahaṁ antar...,ඒකං සමයං භගවා අන්තරා ච රාජගහං අන්තරා ච නාඏඅන්ද...


In [7]:
from pali_text_models import get_engine, create_tables

engine = get_engine()

# Create tables if they don't exist
create_tables(engine)

# Write DataFrame to PostgreSQL
df.to_sql(
    name='sutta_texts',
    con=engine,
    if_exists='replace',  # 'append' to add rows, 'replace' to overwrite
    index=True,
    index_label='index'
)


-1

In [9]:
type(engine)

sqlalchemy.engine.base.Engine

In [12]:
def run_cypher(graph_name: str, cypher_query: str, engine: Engine):
    with Session(engine) as session:
        # AGE requires search_path
        session.exec(text('SET search_path = ag_catalog, "$user", public;'))

        wrapped = f"""
        SELECT * FROM cypher('{graph_name}', $$
        {cypher_query}
        $$) AS (result agtype);
        """

        result = session.exec(text(wrapped))
        rows = result.all()

        return [json.loads(r[0]) for r in rows]

,tablename
0,sutta_texts
